# Chapter 18. 자연어 처리

이번 장에서는 순환 신경망 및 트랜스포머를 이용한 자연어처리(naturla language processing, NLP)를 다룬다.

먼저 플러스의 순환 신경망 구조를 살펴보고, 문자열 생성, 텍스트 분류 등을 실습해 본다. 이어서 트랜스포머 모델 및 허깅페이스 라이브러리도 실습해본다. 

# 18.1 순환 신경망

- 파이토치나 텐서플로의 **순환 신경망**(recurrent neural network, RNN) 모델들은 단일 타임 스텝만 처리하는 **셀**(cell)과 입력 시퀀스를 내분의 반복문으로 한 번에 처리하는 **계층**(layer)으로 구분됨

- 플럭스 역시 순환 신경망에 대해 셀과 계층을 구분한다. 

- 단, 플럭스의 계층은 입력 데이터의 형상에 따라 시간축(시퀀스 길이)이 있으면 셀 호출 반복을 내부적으로 처리하고, 시간축이 없으면 한 스텝만 처리하는 방식이다. 

줄리아는 이썬과 달리 반복문의 성능이 벡터화 이상으로 좋기 대문에 굳이 데이터셋에 시간축을 추가할 필요 없이 명시적인 반복문 안에서 모델을 호출하는 것이 선호됨

## 셀과 래퍼

- 셀에는 `RNNCell`, `LSTMCell`, `GRUCell`이 있고,

- 각각에 대응하는 층으로 `RNN`, `LSTM`, `GRU`가 있다. 

가장 간단한 RNNCell 타입을 살펴보자.

**(참조)** : https://github.com/FluxML/Flux.jl/blob/master/src/layers/recurrent.jl

In [1]:
using Pkg
Pkg.activate("/home/bread/JULIA/chap4")
Pkg.instantiate()

  Activating project at `~/JULIA/chap4`


In [2]:
using Flux

In [3]:
rnn = Flux.RNNCell(2 => 3)
x = rand(Float32, 2); # 입력
h = rand(Float32, 3); # 초기 은닉 상태
h, y = rnn(h, x)      # (다음 은닉 상태, 출력)

(Float32[-0.39472273, -0.63386273, 0.76284176], Float32[-0.39472273, -0.63386273, 0.76284176])

- 층과 달리, 셀은 플럭스에서 export 되지 않으므로 Flux.RNNCell 식으로 사용했다. 

    - RNNCell은 in => out을 생성자의 인수로 받아서 입력 벡터에 곱해지는 (out x in) 사이즈의 가중치 Wi 필드와 은닉 상태(hidden state) 벡터에 곱해지는 (out x out) 사이즈의 가중치 Wh를 필드로 가짐

    - 셀의 호출은 마지막 명령 `h, y = rnn(h, x)`와 같이, 은닉 상태와 입력값을 넘겨서 다음 은닉 상태와 출력값을 돌려 받음

    - 셀 호출 시 x와 h의 각 마지막 차원에 배치 차원을 추가할 수도 있음

셀이 생성하는 은닉 상태를 쉽게 관리하기 위해 플럭스는 `Recur`라는 래퍼(wrapper) 타입을 제공한다.

In [4]:
mutable struct Recur{T, S}
        cell::T
        state::S
end

In [5]:
function (m::Recur)(x)
    m.state, y = m.cell(m.state, x) # 입력값 x로 객체 호출 시 내부 셀에 기존 상태와 x를 넘겨서 상태를 업데이트하고 출력값 y를 리턴함
    return y   
end

In [6]:
using Functors

In [7]:
@functor Recur
trainable(a::Recur) = (; cell = a.cell)   # 앞에서 다뤘던 trainable 함수로 학습 시 은닉 상태의 변경없이 셀의 파라미터만 업데이트 가능 
reset!(m::Recur) = (m.state = m.cell.state0) # reset! 함수로 은닉 상태를 셀의 초기 상태값으로 리셋

reset! (generic function with 1 method)

Recur 타입은 cell과 은닉 상태 벡터인 state 필드를 가지고 있다. 

입력값 x로 객체 호출 시 내부 셀에 기존 상태와 x를 넘겨서 상태를 업데이트하고 출력값 y를 리턴한다.

앞에서 다뤘던 trainable 함수로 학습 시 은닉 상태의 변경 없이 셀의 파라미터만 업데이트할 수 있고,

reset! 함수로 은닉 상태를 셀의 초기 상태값으로 리셋할 수 있다. 

## 계층

RNN, LSTM, GRU는 각각 해당 셀을 래핑한 Recur 객체를 반환하는 함수

플럭스 깃허브에서 가져온 RNN 함수의 소스 코드는 다음과 같다.

In [8]:
flip(f, xs) = reverse([f(x) for x in reverse(xs)])

function (m::Recur)(x::AbstractArray{T, 3}) where T
  h = [m(x_t) for x_t in eachlastdim(x)]
  sze = size(h[1])
  reshape(reduce(hcat, h), sze[1], sze[2], length(h))
end

# Vanilla RNN

struct RNNCell{F,I,H,V,S}
  σ::F
  Wi::I
  Wh::H
  b::V
  state0::S
end

RNNCell((in, out)::Pair, σ=tanh; init=Flux.glorot_uniform, initb=zeros32, init_state=zeros32) =
  RNNCell(σ, init(out, in), init(out, out), initb(out), init_state(out,1))

function (m::RNNCell{F,I,H,V,<:AbstractMatrix{T}})(h, x::AbstractVecOrMat) where {F,I,H,V,T}
  Wi, Wh, b = m.Wi, m.Wh, m.b
  _size_check(m, x, 1 => size(Wi,2))
  σ = NNlib.fast_act(m.σ, x)
  xT = _match_eltype(m, T, x)
  h = σ.(Wi*xT .+ Wh*h .+ b)
  return h, reshape_cell_output(h, x)
end

@functor RNNCell

function Base.show(io::IO, l::RNNCell)
  print(io, "RNNCell(", size(l.Wi, 2), " => ", size(l.Wi, 1))
  l.σ == identity || print(io, ", ", l.σ)
  print(io, ")")
end

RNN(a...; ka...) = Recur(RNNCell(a...; ka...))
Recur(m::RNNCell) = Recur(m, m.state0)

Recur

In [13]:
# 위에서 정의한 사용자 정의 함수로서 RNN 실행 모습

m = RNN(2 => 3)

Recur{RNNCell{typeof(tanh), Matrix{Float32}, Matrix{Float32}, Vector{Float32}, Matrix{Float32}}, Matrix{Float32}}(RNNCell(2 => 3, tanh), Float32[0.0; 0.0; 0.0;;])

In [14]:
# Flux의 RNN 함수 사용하는 경우

Flux.RNN(2 => 3)

Recur(
  RNNCell(2 => 3, tanh),                # 21 parameters
)         # Total: 4 trainable arrays, 21 parameters,
          # plus 1 non-trainable, 3 parameters, summarysize 316 bytes.

위 코드에서 확인되는 바와 같이, RNN은 (RNNCell, 현재 은닉 벡터)로 구성된 Recur 타입이고,

Recur 타입은 입력값 x로 호출 시 은닉 상태를 업데이트하면서 출력값 y를 반환하므로 

(입력 차원 x 배치 크기)를 원소로 가지는 입력 시퀀스에 대해 RNN을 반복 적용하면 다음과 같이 각 타임 스텝별 출력을 얻을 수 있다.

In [23]:
# 위에서 local 환경에 RNN 사용자 함수를 먼저 잡아 놓았기 때문에, Flux.RNN으로 사용

using Flux
using Random

seed_value = 1
Random.seed!(seed_value)

batch_size = 4;
seq_length = 5;
model = Chain(Flux.RNN(2 => 3), Dense(3 => 1));
xs = [rand(Float32, 2, batch_size) for _ = 1:seq_length]; # 행렬의 배열 xs
[model(x) for x in xs]

5-element Vector{Matrix{Float32}}:
 [-0.44558612 -0.72997475 -0.63702464 -0.42720002]
 [-0.87111557 -0.6645896 -0.38225025 -0.18847342]
 [-0.61864096 -0.57454276 -0.95116127 -1.0056882]
 [-0.496481 -0.8767482 -0.78796744 -0.8240887]
 [-0.9099736 -0.70921683 -0.62509054 -0.64954776]

명시적인 반복문 대신 시퀀스를 한 번에 넘기려면 시간축을 마지막 축으로 하여 (입력 차원 x 배치 크기 x 시퀀스 길이)로 넘기면 된다.

다음은 시퀀스를 한 번에 넘기는 예이다.

In [25]:
using MLUtils
Flux.reset!(model) # 모델의 은닉 상태 벡터 초기화
model(MLUtils.batch(xs)) # batch 함수로 행렬의 배열인 xs를 3차원 배열로 만들어 넘김

1×4×5 Array{Float32, 3}:
[:, :, 1] =
 -0.445586  -0.729975  -0.637025  -0.4272

[:, :, 2] =
 -0.871116  -0.66459  -0.38225  -0.188473

[:, :, 3] =
 -0.618641  -0.574543  -0.951161  -1.00569

[:, :, 4] =
 -0.496481  -0.876748  -0.787967  -0.824089

[:, :, 5] =
 -0.909974  -0.709217  -0.625091  -0.649548

모델의 은닉 상태 벡터를 초기화한 후, batch 함수로 행렬의 배열인 xs를 3차원 배열로 만들어 모델에 넘겼다.

그 결과는 명시적인 반복문으로 적용한 결과와 형상만 다를 뿐 값은 같은 것을 볼 수 있다. 

- 시퀀스 단위의 입출력 형상은 파이토치의 RNN 계층에서 batch_first = False로 지정한 경우의 역순과 동일하다. 

- 시간축을 벡터화하는 대신 명시적인 반복문을 이용하면 데이터 조작에 드는 수고도 줄이고 메모리 사용량도 줄일 수 있는 경우가 많다. 

  - 예를 들어 슬라이딩 윈도(sliding window) 방식으로 시계열을 분석하는 경우, 시간축을 벡터화하기 위해 입력 데이터를 ((시계열 길이 - 윈도 크기 + 1) x 윈도 크기) 형태로 새로 만드는 경우가 많다. 

  - 반면 반복문과 view 함수를 이용하면 원 시계열 데이터를 그대로 이용할 수 있다. 

## 손실 함수 정의

각 타임 스텝에서의 출력값과 그 시점의 타깃값을 비교하는 다대다(many-to-many) 모델의 손실함수는 다음과 같이 개별 스텝에서의 손실 합으로 정의 가능

In [26]:
function loss_fn(loss, model, xs, ys)
    sum(loss(model(x), y) for (x, y) in zip(xs, ys))
end

loss_fn (generic function with 1 method)

반면 시퀀스의 마지막 출력값에 최종 타깃값을 비교하는 다대일(many-to-one) 모델의 손실 함수는 다음과 같이 마지막 직전 스텝까지는 은닉 상태만 업데이트하고 마지막에만 손실을 계산하면 된다.

In [27]:
function loss_fn(loss, model, xs, y)
    [m(x) for x in xs[1:end-1]]
    loss(model(x[end]), y)
end

loss_fn (generic function with 1 method)

# 18.2 문자열 생성

이번 절에서는 문자 기반 순환 신경망을 이용하여 문자열을 생성함 

주어진 문자열의 다음 문자를 예측하도록 모델을 학습시켜서, 모델이 원 텍스트와 유사하게 보이는 텍스트를 생성하는 것이 목표이다. 

학습 데이터 셋은 **타이니 셰익스피어 데이터셋**(Tiny Shakespeare)이고, 관련 연구는 안드레이 카파시의 블로그에서 확인할 수 있다.

- (블로그 주소) https://karpathy.github.io/2015/05/21/rnn-effectiveness/

- (데이터 셋 파일 주소) https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [28]:
using Flux
using Flux.Losses: logitcrossentropy
import Zygote, Optimisers
using MLUtils: chunk, batchseq
using StatsBase: wsample
using Formatting: printfmtln
using Random: MersenneTwister

In [29]:
pwd()

"/home/bread/JULIA/chap4"

In [30]:
fpath = "tinyshakespeare.txt"
isfile(fpath)

true

## 데이터 셋 생성

큰 시계열 데이터로 순환 신경망을 학습 시, 역전파를 너무 길게 하면 학습이 오래 걸리므로 적당한 시퀀스 길이로 잘라주는게 좋다.

- 역전파의 연결은 시퀀스 단위로 끊어주지만, 

- 순전파의 경우 이번 시퀀스의 마지막 은닉 상태를 다음 시퀀스의 초기 은닉 상태로 연결하려면 시퀀스 간의 순서가 유지되어야 한다. 


미니배치 학습이 가능하면서 시퀀스 간의 순서를 유지하기 위해 다음과 같이 데이터셋을 구성한다.

1. chunk 함수로 주어진 텍스트를 배치 크기로 등분한다. 

2. batchseq 함수로 배치 리스트를 만든다. 

3. chunk 함수로 배치 리스트를 시퀀스 길이 단위로 묶는다. 

- chunk 함수는 등분되는 개수를 지정함. 두번째 chunk 함수는 size 키워드 인수로 등분되는 파트의 크기를 지정함

- 먼저 전체를 배치 크기로 등분 시, 마지막 파트는 길이가 짧은 수 있고, 이 경우 batchseq 함수 적용 시 끝에 패드(pad)를 추가한다. 

- 세번째 단계에서 시퀀스 길이 단위로 묶을 시에도 마지막 시퀀스는 짧을 수 있다. 

다음은 셰익스피어 텍스트 파일을 읽어서 문자별로 원핫 인코딩을 한 후, 앞의 방식대로 입력 데이터셋과 타깃 데이터셋을 생성하는 함수이다.

In [33]:
function get_data(fpath; batch_size = 32, seq_len = 100)
    text = collect(String(read(fpath)))
    alphabet = unique(text) # 문자 단위 모델이므로 임베딩층을 사용하지 않고 원핫 인코딩을 한다. 
    '_' in alphabet || push!(alphabet, '_'); # 문자 `'_'`는 batchseq 함수에서 필요한 패딩 값으로 사용한다.
    
    text = map(ch -> Flux.onehot(ch, alphabet), text)
    pad = Flux.onehot('_', alphabet);
    
    Xs = batchseq(chunk(text, batch_size), pad) # 1), 2) 단계로 배치 리스트까지 만든다. 
    Xs = map(x -> hcat(x...), Xs) # 3) 단계 적용 전에 각 배치에 hcat 함수를 적용하여 원핫 벡터를 feature 수 x batch 형태의 행렬로 바꿔줌
    Xs = chunk(Xs; size = seq_len) # 3) 단계 적용
    
    Ys = batchseq(chunk(text[2:end], batch_size), pad) # 타겟 데이터셋 Ys는 바로 다음 문자이므로 text[2:end]에 대해 입력 데이터셋과 동일한 방식으로 데이터셋을 만들어 줌
    Ys = map(y -> hcat(y...), Ys)
    Ys = chunk(Ys; size = seq_len)
    
    zip(Xs, Ys), alphabet 
    
    # Xs와 Ys는 데이터 로더를 반복하여 읽는 방식으로 사용할 수 있게 zip 함수로 묶어줌
    # 원핫 인코딩에 사용된 alphabet은 나중에 문자열 생성 시 원핫 벡터를 바꿀 때 사용하기 위해 데이터 셋과 함께 리턴함
end

get_data (generic function with 1 method)

In [42]:
```
function get_data(fpath; batch_size = 32, seq_len = 100)    
    text = collect(String(read(fpath)))
    alphabet = unique(text)
    '_' in alphabet || push!(alphabet, '_');
    
    text = map(ch -> Flux.onehot(ch, alphabet), text)
    pad = Flux.onehot('_', alphabet);

    Xs = batchseq(chunk(text, batch_size), pad)
    Xs = map(x -> hcat(x...), Xs)
    Xs = chunk(Xs; size = seq_len)#[1:end-1];

    Ys = batchseq(chunk(text[2:end], batch_size), pad)
    Ys = map(y -> hcat(y...), Ys)
    Ys = chunk(Ys; size = seq_len)#[1:end-1]

    zip(Xs, Ys), alphabet
end
```

get_data (generic function with 1 method)

문자 단위 모델이므로 임베딩층을 사용하지 않고 원핫 인코딩을 한다. 

원핫 벡터 하나가 그림 18-1의 작은 칸 하나가 된다. 문자 `'_'`는 batchseq 함수에서 필요한 패딩 값으로 사용한다.

1), 2) 단계로 배치 리스트까지 만든 후, 3) 단계 적용 전에 각 배치(네 칸 묶음)에 hcat 함수를 적용하여 원핫 벡터를 (features x batch) 형태의 행렬로 바꿔준다. 

타깃 데이터셋 Ys는 바로 다음 문자이므로 text[2:end]에 대해 입력 데이터셋과 동일한 방식으로 데이터셋을 만들어준다. 

Xs와 Ys는 데이터로더를 반복하여 읽는 방식으로 사용할 수 있게 zip 함수로 묶어준다. 

원핫 인코딩에 사용된 alphabet은 나중에 문자열 생성 시 원핫 벡터를 바꿀 때 사용하기 위해 데이터 셋과 함께 리턴한다. 

In [36]:
loader, alphabet = get_data(fpath, batch_size = 32, seq_len = 100);

배치 크기는 32, 시퀀스 길이는 100으로 데이터셋을 생성함

In [40]:
# 학습 함수 => 16장과 동일

function train(loader, model, loss_fn, optimizer)
    num_batches = length(loader)
    Flux.testmode!(model, false)
    for (batch, (X, y)) in enumerate(loader)
        X, y = Flux.gpu(X), Flux.gpu(y)
        grad = Zygote.gradient(m -> loss_fn(m, X, y), model)[1]
        optimizer, model = Optimisers.update(optimizer, model, grad)
        if batch % 100 == 0
            loss = loss_fn(model, X, y)
            printfmtln("[Train] loss: {:.7f} [{:>3d}/{:>3d}]", 
                loss, batch, num_batches)
        end
    end
    model, optimizer
end
     

train (generic function with 1 method)

## 모델 및 손실 함수

In [37]:
init(rng) = Flux.glorot_uniform(rng)

init (generic function with 1 method)

In [38]:
function build_model(N; rng)
    Chain(
        LSTM(N, 512; init = init(rng)),
        LSTM(512, 512; init = init(rng)),
        Dense(512, N; init = init(rng)))
end;

In [39]:
using Random

seed_value = 1
Random.seed!(seed_value)

rng = MersenneTwister(1);
model = build_model(length(alphabet); rng = rng) |> gpu;
loss_fn(m, xs, ys) = sum(logitcrossentropy.([m(x) for x in xs], ys));
optimizer = Optimisers.setup(Optimisers.Adam(), model);

- 모델은 원핫 벡터의 크기의 입력을 받아서 LSTM 층 두 번을 거친 후 다시 원핫 벡터 크기의 로짓을 출력함

- 손실 함수는 매 스텝의 손실을 합하는 다대다 방식으로 계산

- 정의한 모델과 손실함수, 옵티마이저를 이용하여 다음과 같이 20에폭을 학습

    - 신경망 가중치 초기화에 사용하는 init 함수 및 학습 시 사용하는 train 함수는 16.2절 및 16.3절에서 각각 정의한 동명의 함수들을 그대로 사용함

In [44]:
```
init(rng) = Flux.glorot_uniform(rng)

function build_model(N; rng)
    Chain(
        LSTM(N, 512; init=init(rng)),
        LSTM(512, 512; init=init(rng)),
        Dense(512, N; init=init(rng))
    )
end;

rng = MersenneTwister(1)
model = build_model(length(alphabet); rng=rng) |> gpu;
loss_fn(m, xs, ys) = sum(logitcrossentropy.([m(x) for x in xs], ys));
optimizer = Optimisers.setup(Optimisers.Adam(), model);
```

In [46]:
@time begin
    for epoch in 1:20
        Flux.reset!(model)
        println("Epoch $epoch")
        println("-------------------------------")
        global model, optimizer = train(loader, model, loss_fn, optimizer)
    end
end

Epoch 1
-------------------------------
[Train] loss: 207.4382935 [100/349]
[Train] loss: 196.8038483 [200/349]
[Train] loss: 188.2085876 [300/349]
Epoch 2
-------------------------------
[Train] loss: 184.0833588 [100/349]
[Train] loss: 176.9190979 [200/349]
[Train] loss: 169.2527313 [300/349]
Epoch 3
-------------------------------
[Train] loss: 170.0822754 [100/349]
[Train] loss: 165.1187439 [200/349]
[Train] loss: 158.6803894 [300/349]
Epoch 4
-------------------------------
[Train] loss: 160.2009277 [100/349]
[Train] loss: 157.0136414 [200/349]
[Train] loss: 151.0191193 [300/349]
Epoch 5
-------------------------------
[Train] loss: 153.5201874 [100/349]
[Train] loss: 151.1076965 [200/349]
[Train] loss: 145.3099670 [300/349]
Epoch 6
-------------------------------
[Train] loss: 148.1502533 [100/349]
[Train] loss: 146.4346924 [200/349]
[Train] loss: 140.4650269 [300/349]
Epoch 7
-------------------------------
[Train] loss: 144.6532288 [100/349]
[Train] loss: 142.8794250 [200/349]


- 매 에폭 시작마다 Flux.reset! 함수로 순환 신경망 모델의 은닉 상태를 초기화한다. 

- 한 에폭 안에서 미니배치 내 각 시퀀스 별로 시퀀스의 시작 시점에 이전 시퀀스의 마지막 은닉 상태를 이용한다. 

- 이러한 순환 신경망을 상태가 있는(stateful) 순환 신경망이라고 한다. 

- 만약 매 시퀀스 시작마다 은닉 상태를 초기화하는 상태가 없는 (stateless) 모델을 사용하려면 

- train 함수 내 미니배치를 도는 반복문 안에서 Flux.reset!으로 은닉 상태를 초기화하면 된다.

In [49]:
versioninfo()

Julia Version 1.9.3
Commit bed2cd540a1 (2023-08-24 14:43 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 20 × 12th Gen Intel(R) Core(TM) i7-12700F
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-14.0.6 (ORCJIT, alderlake)
  Threads: 2 on 20 virtual cores
Environment:
  LD_LIBRARY_PATH = :/usr/local/cuda-11.7/lib64


약 12분 소요됨

## 학습 및 문자열 생성

In [47]:
# 학습된 모델로 가짜 셰익스피어 텍스트를 생성하는 함수를 만듦

function generate(model, alphabet, init, len; rng)
    model = model |> cpu
    Flux.reset!(model)
    generated = [init]
    for _ in 1:len
        w = softmax(model(Flux.onehot(generated[end], alphabet)))
        push!(generated, wsample(rng, alphabet, w))
    end
    text = String(generated)
    for r in split(text, '\n')
        println(r)
    end
    text
end

generate (generic function with 1 method)

- generate 함수는 학습된 모델과 get_data 함수에서 돌려받은 alphabet, 시작 문자, 문자열 길이를 입력받아서 해당 길이의 문자열을 생성

- 주어진 문자를 원핫 벡터로 바꾸고 모델을 통과하여 나온 로짓값에 소프트맥스를 적용하면 다음 문자에 대한 확률을 얻을 수 있음

- wsample 함수를 이용하여 이 확률대로 다음 문자를 샘플링하여 문자 리스트인 generated에 추가하고, 다시 이 문자를 모델에 통과시켜 그다음 문자를 샘들링한다. 

- 이 과정을 거쳐서 생성된 가짜 셰익스피어 텍스트의 예는 다음과 같다. 

In [52]:
generate(model, alphabet, 'O', 500; rng = MersenneTwister(1));

O:
Sir, Kill'd his father.

Provost:
It is prevail'd against your deal disqueste!
What should the marched but a bitter church,
His bust and feak?  By God of such too list another,
And I: thy comfort is not: take my begg'd;
So should that make a sedming formprise of;
And that I here repose thy prisonesselve;
And I'll tell me shall joy enough. Or how
To she doch friends that intend of your brother's,
Having thy libter's string with these bloody
As only suburd my lord's choice must clamb.

KING RICH


lstm을 이용한 문자열 생성.  끝.